In [6]:
import torch

def load_ensemble(filepaths):
    def load_from_checkpoint(filepath,):
        parameters = torch.load(filepath, map_location='cpu')['state_dict']['bayesian_layer'].params
        parameters = {k:v.detach().numpy().tolist() for k,v in parameters.items() if v.numel()>0 and v.shape == (4096, 4096)}
        return parameters
    return [load_from_checkpoint(filepath) for filepath in filepaths]

parameters = load_ensemble(['logs/bayes/checkpoints/epoch=0-step=300.ckpt','logs/bayes/checkpoints/epoch=9-step=3000.ckpt'])

In [7]:
from examples.bayes_llama3.llama3.modules.bayesllama import BayesLlamaForCausalLM

bayes_config = {'n_ensemble':2, 'ensemble_param_layer':-1}
bayes = BayesLlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", bayes_config=bayes_config).to('cuda:0')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
bayes.load(bayes_params = parameters)

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

inputs = "This is a test"
inputs = tokenizer(inputs, return_tensors="pt").input_ids

In [10]:
out = bayes(inputs.to('cuda:0'), return_dict=True)